# *RUN THE FIRST TWO CHUNKS*

In [42]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

#### Import LOB, LOD, LOQ values

In [43]:
rpath = "C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LOD paper\\LOD_calcs.csv"
results = pd.read_csv(rpath)

#### Import compositions

In [44]:
cpath = "C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LANL cal_comps\\LANL_rdr_cal2015_1600mm_pls_metadata_071321.csv"
comps = pd.read_csv(cpath)

# First/reset calculations?
#### Set up results table

In [45]:
full_table = pd.DataFrame({
    "element" : [],
    'instrument' : [],
    'atmosphere' : [],
    'num_range' : [],
    "Avg_Braga" : [],
    "Avg_metals" : [],
    "RMSEP_Braga" : [],
    "RMSEP_metals" : [],
    "R2_Braga" : [],
    "R2_metals" : []
})

# Adding new data?
#### Import existing table

In [46]:
#fpath = "C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LOD paper\\test_calcs.csv"
#full_table = pd.read_csv(fpath)

# \*\*\*Start here with a new calculation\*\*\*
Data should be in a `XX_preds.csv` or `XX_predictions.csv` format within folders separated by random number region (0-750 or 250-1000)

- **LANL 0-750:** C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LOD paper\\LANL calculations\\models\\0-750\\
- **LANL 250-1000:** C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LOD paper\\LANL calculations\\models\\250-1000\\

In [54]:
instrument = input("What instrument was the data collected on? ")
atm = input("What atmosphere was the data collected under? ")
num_range = input("What range of random numbers were the models trained on? (0-750 or 250-1000)")
folder = input('Folder path containing test predictions:')
predlist = os.listdir(folder)

What instrument was the data collected on? LANL
What atmosphere was the data collected under? Mars
What range of random numbers were the models trained on? (0-750 or 250-1000)250-1000
Folder path containing test predictions:C:\Users\ytsma22c\Google Drive\Darby Work\LOD paper\LANL calculations\models\250-1000\


In [55]:
# subselect the relevant parts of the results table
new_results = results[(results.instrument == instrument) & 
                      (results.atmosphere == atm) &
                      (results.num_range == num_range)]

In [56]:
# prepare lists to be filled
# ADD LISTS FOR 'ALL' AND 'ABOVE LOD' CALCULATIONS IF DESIRED
elem = []
avg_braga = []
rmsep_braga = []
r2_braga = []
avg_metal = []
rmsep_metal = []
r2_metal = []

In [57]:
for file in tqdm(predlist):
    if "test" in file:
        #get path name        
        path = (folder + file)
        #read in file
        data = pd.read_csv(path)
        #get element name, add to list
        element_unit = data.columns[1]
        # remove units if they're there
        if "ppm" in element_unit:
            element = element_unit.replace(" (ppm)", "")
        elif "wt%" in element_unit:
            element = element_unit.replace(" (wt%)", "")
        elem.append(element)
        # format columns
        data.columns = ['pkey', 'Actual', 'Pred']
        data = data.drop([0])
        data.Pred = data.Pred.astype(float)       
        # remove predictions above 100 for majors
        if element in ['SiO2', 'MnO', 'Na2O']:
            data = data[data.Pred < 100]
        # remove all predictions below 0
        data = data[data.Pred > 0].reset_index(drop=True)
        # subselect relevant reference values
        ref = new_results[new_results.element == element].reset_index(drop=True)

        # add in Actual concentrations
        temp_comps = comps[comps.pkey.isin(data.pkey)].reset_index(drop=True)
        # remove units
        if "ppm" in element:
            element = element.replace(" (ppm)", "")
        elif "wt%" in element:
            element = element.replace(" (wt%)", "")
        data['Actual'] = temp_comps[temp_comps['pkey'] == data['pkey']][element_unit]

#### COULD ADD CODE HERE TO CALCULATE AVERAGES, RMSE-PS, AND R2S FOR #####
####### ALL VALUES AND FOR THOSE ABOVE THE LOD BUT BELOW THE LOQ #########
        
        ###BRAGA###
        loq_b = ref['LOQ_Braga'].iloc[0]
        # select just predictions above the LOQ
        braga = data[data.Pred > loq_b].reset_index(drop=True)
        # get average concentration
        avg_b = braga['Actual'].mean()
        avg_braga.append(avg_b)
        # get R2
        r2_b = r2_score(braga.Actual, braga.Pred)
        r2_braga.append(r2_b)
        # get RMSE-P
        braga['sqerror'] = (braga.Actual - braga.Pred).pow(2)
        rmse_b = braga['sqerror'].mean() ** 0.5
        rmsep_braga.append(rmse_b)
        
        ##METALS###
        loq_m = ref['LOQ_metals'].iloc[0]
        # select just predictions above the LOQ
        metal = data[data.Pred > loq_m].reset_index(drop=True)
        # get average concentration
        avg_m = metal['Actual'].mean()
        avg_metal.append(avg_m)
        # get R2
        r2_m = r2_score(metal.Actual, metal.Pred)
        r2_metal.append(r2_m)
        # get RMSE-P
        metal['sqerror'] = (metal.Actual - metal.Pred).pow(2)
        rmse_m = metal['sqerror'].mean() ** 0.5
        rmsep_metal.append(rmse_m)

  0%|          | 0/37 [00:00<?, ?it/s]

C:\Users\ytsma22c\Anaconda3\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [58]:
max(temp_comps['Na2O (wt%)'])

6.777895166

#### Show results

In [59]:
instr = [instrument] * len(elem)
atmos = [atm] * len(elem)
num = [num_range] * len(elem)

temp = pd.DataFrame({
    "element" : elem,
    'instrument' : instr,
    'atmosphere' : atmos,
    'num_range' : num,
    "Avg_Braga" : avg_braga,
    "Avg_metals" : avg_metal,
    "RMSEP_Braga" : rmsep_braga,
    "RMSEP_metals" : rmsep_metal,
    "R2_Braga" : r2_braga,
    "R2_metals" : r2_metal
})

display(temp)

,element,instrument,atmosphere,num_range,Avg_Braga,Avg_metals,RMSEP_Braga,RMSEP_metals,R2_Braga,R2_metals
0,Li,LANL,Mars,250-1000,35.000000,32.100000,28.446800,12.274406,NaN,-0.523521
1,MnO,LANL,Mars,250-1000,0.098526,0.098437,0.093383,0.093280,-0.282251,-0.292748
2,Na2O,LANL,Mars,250-1000,2.419960,2.154462,0.709711,0.666996,0.798913,0.835209
3,Ni,LANL,Mars,250-1000,82.845108,80.627450,91.394430,89.920207,0.282803,0.292004
4,Pb,LANL,Mars,250-1000,16.786241,16.527991,16.495796,16.368837,0.178842,0.189404
5,Rb,LANL,Mars,250-1000,97.385494,90.021522,27.756921,26.687355,0.811857,0.838157
6,SiO2,LANL,Mars,250-1000,57.112753,55.644556,4.033074,4.063333,0.890554,0.922536
7,Sr,LANL,Mars,250-1000,444.363967,257.265019,133.327433,95.447464,0.255817,0.706744
8,Zn,LANL,Mars,250-1000,74.670337,64.703057,40.612659,38.105105,0.262361,0.383381


### Add to results table

In [60]:
full_table = full_table.append(temp).reset_index(drop = True)
# drop unit info
full_table.element = full_table.apply(lambda row: row.element.split()[0], axis = 1)
display(full_table)

,element,instrument,atmosphere,num_range,Avg_Braga,Avg_metals,RMSEP_Braga,RMSEP_metals,R2_Braga,R2_metals
0,Li,LANL,Mars,0-750,60.575000,39.068421,9.668296,8.810213,0.458053,0.752752
1,MnO,LANL,Mars,0-750,0.151139,0.151139,0.276825,0.276825,0.028307,0.028307
2,Na2O,LANL,Mars,0-750,2.523204,2.303813,0.643707,0.615807,0.837848,0.863066
3,Ni,LANL,Mars,0-750,66.712091,66.712091,80.429236,80.429236,0.124015,0.124015
4,Pb,LANL,Mars,0-750,8.286433,8.004541,11.920810,11.589361,-0.023132,-0.017396
5,Rb,LANL,Mars,0-750,93.839902,74.434178,41.023144,35.937756,0.598868,0.694694
6,SiO2,LANL,Mars,0-750,56.407547,54.098970,5.143286,5.458626,0.790775,0.867180
7,Sr,LANL,Mars,0-750,374.740455,277.797361,187.959410,145.022088,-0.434960,0.167576
8,Zn,LANL,Mars,0-750,66.520103,64.455599,61.691908,60.590888,0.287224,0.301851
9,Li,LANL,Mars,250-1000,35.000000,32.100000,28.446800,12.274406,NaN,-0.523521


### Export full table

In [61]:
epath = "C:\\Users\\ytsma22c\\Google Drive\\Darby Work\\LOD paper\\test_calcs.csv"
full_table.to_csv(epath, index=False)